In [1]:
!pip install gspread
!pip install oauth2client

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [24]:
import pandas, gspread, random, numpy

In [2]:
#https://medium.com/analytics-vidhya/how-to-read-and-write-data-to-google-spreadsheet-using-python-ebf54d51a72c
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('ai-pm-project-46ef9dbd112c.json', scope)
client = gspread.authorize(credentials)

# Import Google Sheet

In [43]:
# Open a worksheet
sheet = client.open("AI PM Data Structure").sheet1

# Get all records
records = sheet.get_all_records()
# Print the records
print(records)

[{'simulation_start_time': '', 'success_or_failure': '', 'week': 0, 'scope_remaining_at_week_start_index': '', 'scope_remaining_at_week_end_index': 42, 'schedule_remaining_at_week_start_week': '', 'schedule_remaining_at_week_end_week': 21, 'budget_remaining_at_week_start_dollar': '', 'budget_remaining_at_week_end_dollar': '$81,900', 'scope_shock_during_the_week_index': '', 'standard_weekly_labour_rate_dollar': 1500, 'resource_lv1_productivity_during_the_week_index': 0.5, 'resource_lv2_productivity_during_the_week_index': 0.9, 'resource_lv3_productivity_during_the_week_index': 1.15, 'resource_lv1_available_during_the_week_count': 2, 'resource_lv2_available_during_the_week_count': 2, 'resource_lv3_available_during_the_week_count': 2, 'resource_lv1_weekly_rate_dollar': 900, 'resource_lv2_weekly_rate_dollar': 1500, 'resource_lv3_weekly_rate_dollar': 1875, 'resource_lv1_used_during_the_week_count': '', 'resource_lv2_used_during_the_week_count': '', 'resource_lv3_used_during_the_week_count':

In [44]:
simulation_df = pandas.DataFrame(records)
simulation_df.loc[:,:].dtypes

simulation_start_time                                        object
success_or_failure                                           object
week                                                          int64
scope_remaining_at_week_start_index                          object
scope_remaining_at_week_end_index                           float64
schedule_remaining_at_week_start_week                        object
schedule_remaining_at_week_end_week                           int64
budget_remaining_at_week_start_dollar                        object
budget_remaining_at_week_end_dollar                          object
scope_shock_during_the_week_index                            object
standard_weekly_labour_rate_dollar                            int64
resource_lv1_productivity_during_the_week_index             float64
resource_lv2_productivity_during_the_week_index             float64
resource_lv3_productivity_during_the_week_index             float64
resource_lv1_available_during_the_week_count    

# Cleanse dollar columns as they are originally stored as object/str when transferring from Google Sheet

In [45]:
column_with_dollar_list = []
#identify column names with $
for column in simulation_df.columns:
    if 'dollar' in column:
        column_with_dollar_list.append(column)
#replace column values in dataframe
for column in column_with_dollar_list:
    simulation_df[column] = simulation_df[column].astype('str').str.replace('$','').str.replace(',','')
    #empty str '' cannot directly apply astype float. need code below
    simulation_df[column] = pandas.to_numeric(simulation_df[column],errors='coerce').astype('float')
simulation_df[column_with_dollar_list]

,budget_remaining_at_week_start_dollar,budget_remaining_at_week_end_dollar,standard_weekly_labour_rate_dollar,resource_lv1_weekly_rate_dollar,resource_lv2_weekly_rate_dollar,resource_lv3_weekly_rate_dollar,resouce_lv1_actual_cost_during_the_week_dollar,resouce_lv2_actual_cost_during_the_week_dollar,resouce_lv3_actual_cost_during_the_week_dollar
0,NaN,81900.0,1500.0,900.0,1500.0,1875.0,0.0,0.0,0.0
1,81900.0,78900.0,1500.0,900.0,1500.0,1875.0,0.0,3000.0,0.0
2,78900.0,75900.0,1500.0,900.0,1500.0,1875.0,0.0,3000.0,0.0


# Simulation set up

In [ ]:
import random
from datetime import datetime



#variable for storing simulation result
result_df = simulation_df.loc[0:0,:]

number_of_simulations = 500

#loop multiple simulations
for simulation in range(0,number_of_simulations,1):
    #initialization based on row 0 or week 0
    scope_remaining = int(simulation_df.loc[0,'scope_remaining_at_week_end_index'])
    schedule_initial = int(simulation_df.loc[0,'schedule_remaining_at_week_end_week'])
    schedule_remaining = schedule_initial
    budget_remaining = int(simulation_df.loc[0,'budget_remaining_at_week_end_dollar'])
    scope_shock_during_the_week_index = 0
    standard_weekly_labour_rate_dollar = float(simulation_df.loc[0,'standard_weekly_labour_rate_dollar'])
    resource_lv1_productivity_during_the_week_index = float(simulation_df.loc[0,'resource_lv1_productivity_during_the_week_index'])
    resource_lv2_productivity_during_the_week_index = float(simulation_df.loc[0,'resource_lv2_productivity_during_the_week_index'])
    resource_lv3_productivity_during_the_week_index = float(simulation_df.loc[0,'resource_lv3_productivity_during_the_week_index'])
    resource_lv1_available_during_the_week_count = float(simulation_df.loc[0,'resource_lv1_available_during_the_week_count'])
    resource_lv2_available_during_the_week_count = float(simulation_df.loc[0,'resource_lv2_available_during_the_week_count'])
    resource_lv3_available_during_the_week_count = float(simulation_df.loc[0,'resource_lv3_available_during_the_week_count'])
    resource_lv1_weekly_rate_dollar = float(simulation_df.loc[0,'resource_lv1_weekly_rate_dollar'])
    resource_lv2_weekly_rate_dollar = float(simulation_df.loc[0,'resource_lv2_weekly_rate_dollar'])
    resource_lv3_weekly_rate_dollar = float(simulation_df.loc[0,'resource_lv3_weekly_rate_dollar'])

    result_df_last_row_number = result_df.shape[0]
    simulation_start_time = datetime.now()
    success_or_failure = '' 
    
    #each simulation starts from week 1
    for week_num in range(1,schedule_initial+1,1):
        #week starting condition based on previous week ending condition
        schedule_remaining_at_week_start_week = schedule_remaining
        scope_remaining_at_week_start_index = scope_remaining
        budget_remaining_at_week_start_dollar = budget_remaining
        
        #parameters for this week. keep constant for now
        scope_shock_during_the_week_index
        standard_weekly_labour_rate_dollar
        resource_lv1_productivity_during_the_week_index
        resource_lv2_productivity_during_the_week_index
        resource_lv3_productivity_during_the_week_index
        resource_lv1_available_during_the_week_count
        resource_lv2_available_during_the_week_count
        resource_lv3_available_during_the_week_count
        resource_lv1_weekly_rate_dollar
        resource_lv2_weekly_rate_dollar
        resource_lv3_weekly_rate_dollar
        #update schedule    
        schedule_remaining_at_week_end_week = schedule_remaining - 1
        schedule_remaining = schedule_remaining_at_week_end_week
        
        #make decisions
        resource_lv1_used_during_the_week_count = random.randint(0,resource_lv1_available_during_the_week_count)
        resource_lv2_used_during_the_week_count = random.randint(0,resource_lv2_available_during_the_week_count)
        resource_lv3_used_during_the_week_count = random.randint(0,resource_lv3_available_during_the_week_count)  
        
       
        
        #update scope remaining
        resource_lv1_actual_accomplishment_during_the_week_index = resource_lv1_productivity_during_the_week_index * resource_lv1_used_during_the_week_count
        resource_lv2_actual_accomplishment_during_the_week_index = resource_lv2_productivity_during_the_week_index * resource_lv2_used_during_the_week_count
        resource_lv3_actual_accomplishment_during_the_week_index = resource_lv3_productivity_during_the_week_index * resource_lv3_used_during_the_week_count
        scope_remaining_at_week_end_index = (
            scope_remaining_at_week_start_index + scope_shock_during_the_week_index 
            - resource_lv1_actual_accomplishment_during_the_week_index 
            - resource_lv2_actual_accomplishment_during_the_week_index 
            - resource_lv3_actual_accomplishment_during_the_week_index)
        scope_remaining = scope_remaining_at_week_end_index
        #update budget remaining
        resouce_lv1_actual_cost_during_the_week_dollar = resource_lv1_weekly_rate_dollar * resource_lv1_used_during_the_week_count
        resouce_lv2_actual_cost_during_the_week_dollar = resource_lv2_weekly_rate_dollar * resource_lv2_used_during_the_week_count
        resouce_lv3_actual_cost_during_the_week_dollar = resource_lv3_weekly_rate_dollar * resource_lv3_used_during_the_week_count
        budget_remaining_at_week_end_dollar = (
            budget_remaining_at_week_start_dollar
            - resouce_lv1_actual_cost_during_the_week_dollar
            - resouce_lv2_actual_cost_during_the_week_dollar
            - resouce_lv3_actual_cost_during_the_week_dollar)
        budget_remaining = budget_remaining_at_week_end_dollar
    
        #determine success or failure by looking at week end status
        if (scope_remaining_at_week_end_index<=0
            and schedule_remaining_at_week_end_week >=0
            and budget_remaining_at_week_end_dollar >=0):
                success_or_failure = 'success'
        elif (schedule_remaining_at_week_end_week < 0
              or (schedule_remaining_at_week_end_week == 0 and scope_remaining_at_week_end_index>0)
              or budget_remaining_at_week_end_dollar < 0):
                success_or_failure = 'failure'
        #else: keep going
        
        #write to the file
        new_row = {'simulation_start_time':simulation_start_time
                   ,'success_or_failure':success_or_failure
                   ,'week':week_num
                   ,'scope_remaining_at_week_start_index':scope_remaining_at_week_start_index
                   ,'scope_remaining_at_week_end_index':scope_remaining_at_week_end_index
                   ,'schedule_remaining_at_week_start_week':schedule_remaining_at_week_start_week
                   ,'schedule_remaining_at_week_end_week':schedule_remaining_at_week_end_week
                   ,'budget_remaining_at_week_start_dollar':budget_remaining_at_week_start_dollar
                   ,'budget_remaining_at_week_end_dollar':budget_remaining_at_week_end_dollar
                   ,'scope_shock_during_the_week_index':scope_shock_during_the_week_index
                   ,'standard_weekly_labour_rate_dollar':standard_weekly_labour_rate_dollar
                   ,'resource_lv1_productivity_during_the_week_index':resource_lv1_productivity_during_the_week_index
                   ,'resource_lv2_productivity_during_the_week_index':resource_lv2_productivity_during_the_week_index
                   ,'resource_lv3_productivity_during_the_week_index':resource_lv3_productivity_during_the_week_index
                   ,'resource_lv1_available_during_the_week_count':resource_lv1_available_during_the_week_count
                   ,'resource_lv2_available_during_the_week_count':resource_lv2_available_during_the_week_count
                   ,'resource_lv3_available_during_the_week_count':resource_lv3_available_during_the_week_count
                   ,'resource_lv1_weekly_rate_dollar':resource_lv1_weekly_rate_dollar
                   ,'resource_lv2_weekly_rate_dollar':resource_lv2_weekly_rate_dollar
                   ,'resource_lv3_weekly_rate_dollar':resource_lv3_weekly_rate_dollar
                   ,'resource_lv1_used_during_the_week_count':resource_lv1_used_during_the_week_count
                   ,'resource_lv2_used_during_the_week_count':resource_lv2_used_during_the_week_count
                   ,'resource_lv3_used_during_the_week_count':resource_lv3_used_during_the_week_count
                   ,'resource_lv1_actual_accomplishment_during_the_week_index':resource_lv1_actual_accomplishment_during_the_week_index
                   ,'resource_lv2_actual_accomplishment_during_the_week_index':resource_lv2_actual_accomplishment_during_the_week_index
                   ,'resource_lv3_actual_accomplishment_during_the_week_index':resource_lv3_actual_accomplishment_during_the_week_index
                   ,'resouce_lv1_actual_cost_during_the_week_dollar':resouce_lv1_actual_cost_during_the_week_dollar
                   ,'resouce_lv2_actual_cost_during_the_week_dollar':resouce_lv2_actual_cost_during_the_week_dollar
                   ,'resouce_lv3_actual_cost_during_the_week_dollar':resouce_lv3_actual_cost_during_the_week_dollar
                  }
        result_df.loc[result_df_last_row_number+week_num] = new_row
    
        #if either success or failure occurs, stop the simulation
        if success_or_failure == 'success' or success_or_failure == 'failure':
            break
        #simulation ends


#print((pandas.DataFrame(result_df)))
#print(success_or_failure)


result_df.to_csv('simulation:'+str(datetime.now())+'.csv', index=False)

/tmp/ipykernel_118/2895596265.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.loc[result_df_last_row_number+week_num] = new_row


In [47]:
pandas.DataFrame({'simulation_start_time':[None]})

,simulation_start_time
0,None


In [84]:
range(1,number_of_simulations,1)

range(1, 1)